## Bad vs Good Connections
### Author: Ana Javed

You are involved in a project where you are tasked to build a machine learning algorithm that distinguishes between "bad'' connections (called intrusions or attacks) and "good'' (normal) connections. Note that the number of normal connections is greater than that of bad ones. 

The dataset you will use in this assignment originated with the KDD Cup 1999 (https://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html).


In [1]:
## Importing Necessary Libraries & Packages 
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import csv
import sklearn 

from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


In [2]:
## Reading data file into Dataframe 
url = "https://library.startlearninglabs.uw.edu/DATASCI420/2019/Datasets/Intrusion%20Detection.csv"
df = pd.read_csv(url, sep=",")

## Top 5 Rows from Dataframe
df.head()


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Class
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,0
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0


In [3]:
## Conducting Exploratory Data Analysis: 
print(df.shape)  # (97308, 42) 
print(df.dtypes) # many int64, object, and float64 types 
print(df.describe()) 

(97308, 42)
duration                         int64
protocol_type                   object
service                         object
flag                            object
src_bytes                        int64
dst_bytes                        int64
land                             int64
wrong_fragment                   int64
urgent                           int64
hot                              int64
num_failed_logins                int64
logged_in                        int64
num_compromised                  int64
root_shell                       int64
su_attempted                     int64
num_root                         int64
num_file_creations               int64
num_shells                       int64
num_access_files                 int64
num_outbound_cmds                int64
is_host_login                    int64
is_guest_login                   int64
count                            int64
srv_count                        int64
serror_rate                    float64
srv_serror_ra

In [4]:
## Obtaining a list of Column Names that will be Used for Normalization & Dummy Variables 
other_columns = list(df.columns)
# Removing the Target Variable 
other_columns.remove('Class')
print(other_columns)

## Creating Dummy Variables for Categorical Fields 
df_expanded = pd.get_dummies(df.loc[:, other_columns], drop_first = True)



['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']


In [5]:
## Standardizing / Normalizing Values with StandardScaler 
standardization_scale = StandardScaler().fit(df_expanded)
normalized_df = pd.DataFrame(standardization_scale.transform(df_expanded), columns=df_expanded.columns)


In [6]:
## Double Checking Output of Data, to ensure it appears correct

print(normalized_df.shape) #(97308, 72)
normalized_df

(97308, 72)


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,service_urh_i,service_urp_i,flag_REJ,flag_RSTO,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF
0,-0.159396,-0.028524,0.054945,-0.003206,0.0,-0.003206,-0.052515,-0.008904,0.625084,-0.007165,...,-0.011996,-0.074493,-0.240988,-0.026444,-0.017852,-0.022899,-0.023564,-0.013219,-0.008482,0.246407
1,-0.159396,-0.026829,-0.077172,-0.003206,0.0,-0.003206,-0.052515,-0.008904,0.625084,-0.007165,...,-0.011996,-0.074493,-0.240988,-0.026444,-0.017852,-0.022899,-0.023564,-0.013219,-0.008482,0.246407
2,-0.159396,-0.026946,-0.054522,-0.003206,0.0,-0.003206,-0.052515,-0.008904,0.625084,-0.007165,...,-0.011996,-0.074493,-0.240988,-0.026444,-0.017852,-0.022899,-0.023564,-0.013219,-0.008482,0.246407
3,-0.159396,-0.027414,-0.054522,-0.003206,0.0,-0.003206,-0.052515,-0.008904,0.625084,-0.007165,...,-0.011996,-0.074493,-0.240988,-0.026444,-0.017852,-0.022899,-0.023564,-0.013219,-0.008482,0.246407
4,-0.159396,-0.027472,-0.036025,-0.003206,0.0,-0.003206,-0.052515,-0.008904,0.625084,-0.007165,...,-0.011996,-0.074493,-0.240988,-0.026444,-0.017852,-0.022899,-0.023564,-0.013219,-0.008482,0.246407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97303,0.076807,0.010195,-0.039884,-0.003206,0.0,-0.003206,-0.052515,-0.008904,0.625084,-0.007165,...,-0.011996,-0.074493,-0.240988,37.815341,-0.017852,-0.022899,-0.023564,-0.013219,-0.008482,-4.058327
97304,-0.126283,0.034449,0.021703,-0.003206,0.0,-0.003206,3.438021,-0.008904,0.625084,0.239971,...,-0.011996,-0.074493,-0.240988,-0.026444,-0.017852,-0.022899,-0.023564,-0.013219,-0.008482,0.246407
97305,-0.029889,0.011744,-0.017395,-0.003206,0.0,-0.003206,3.438021,-0.008904,0.625084,0.981379,...,-0.011996,-0.074493,-0.240988,-0.026444,-0.017852,-0.022899,-0.023564,-0.013219,-0.008482,0.246407
97306,-0.114510,0.034449,0.021517,-0.003206,0.0,-0.003206,3.438021,-0.008904,0.625084,0.239971,...,-0.011996,-0.074493,-0.240988,-0.026444,-0.017852,-0.022899,-0.023564,-0.013219,-0.008482,0.246407


In [7]:
## Splitting Training and Testing Data
x_train, x_test, y_train, y_test = train_test_split (np.array(normalized_df),
                                                     np.array(df.loc[:,"Class"]),
                                                     test_size = 0.5,
                                                     random_state = 123)


In [8]:
## Creating pipeline to conducting Recursive Feature Elimination (RFE) first 
## And then apply the Decision Tree Classifier
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select= 5)
model = DecisionTreeClassifier()
pipeline = Pipeline(steps=[('s',rfe), ('m',model)])

In [9]:
# Fitting the model on all training data
pipeline.fit(x_train, y_train)

Pipeline(steps=[('s',
                 RFE(estimator=DecisionTreeClassifier(),
                     n_features_to_select=5)),
                ('m', DecisionTreeClassifier())])

In [10]:
## Using the model to predict class with Test Data 
predictions = pipeline.predict(x_test)
target = y_test # Actual 

# Mean Accuracy of Predictions 
print("Mean Accuracy: ", pipeline.score(x_test, y_test)) 

# print(predictions, target)
# print(sum(predictions), sum(target)) #13, 19


Mean Accuracy:  0.9997944670530686


In [11]:
# Confusion Matrix from your predicted values 
from sklearn.metrics import *

# Passing Target and Prediction Values to Confusion_Matrix
CM = confusion_matrix(target, predictions)
print ("\n\nConfusion matrix:\n", CM)

tn, fp, fn, tp = CM.ravel()
print ("\nTP, TN, FP, FN:", tp, ",", tn, ",", fp, ",", fn)


P = precision_score(target, predictions)
print ("\nPrecision Score:", np.round(P, 2))
print (' ')
R = recall_score(target, predictions)
print ("\nRecall Score:", np.round(R, 2))
print (' ')
F1 = f1_score(target, predictions)
print ("\nF1 Score:", np.round(F1, 2))
print (' ')
a = accuracy_score(target, predictions)
print ("\nAccuracy Score:", np.round(a, 2))
print (' ')





Confusion matrix:
 [[48629     6]
 [    4    15]]

TP, TN, FP, FN: 15 , 48629 , 6 , 4

Precision Score: 0.71
 

Recall Score: 0.79
 

F1 Score: 0.75
 

Accuracy Score: 1.0
 


#### Summary for Part 1 
The confusion matrix metrics before resampling data shows okay results. Precision score - which is a good measure of reliability of the classifiers predictions - is at 71%. The Recall score - which estiamtes how well a classifier predicted a specific class - is slightly higher at 79%. These are good amounts but they can certainly be higher. In addition, the accuracy score is not necessarily reliable since there is a class imbalance in the dataset (only 19 total negative values compared to the total 97308 data points). We will revisit the accuracy score and see if it changes after handling the class imbalance below. 

### Part 2: Modifying data by handling class imbalance

In [12]:
## Loading SMOTE Package 
from imblearn.over_sampling import SMOTE 

sm = SMOTE(random_state= 123)
X_res, y_res = sm.fit_resample(x_train, y_train)

In [13]:
## Splitting Training and Testing Data with Resampled Datasets 
x_train, x_test, y_train, y_test = train_test_split (X_res,
                                                     y_res,
                                                     test_size = 0.5,
                                                     random_state = 123)

# Creating the pipeline with the Resampled Datasets 
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select= 5)
model = DecisionTreeClassifier()
pipeline = Pipeline(steps=[('s',rfe), ('m',model)])

# Fitting the model on training data
pipeline.fit(x_train, y_train)



Pipeline(steps=[('s',
                 RFE(estimator=DecisionTreeClassifier(),
                     n_features_to_select=5)),
                ('m', DecisionTreeClassifier())])

In [25]:
## Checking the Total Values
uni, freq = np.unique(y_res, return_counts = True)  # 50% Class 0, #50% Class 1
print(len(y_res), uni, freq )

print(X_res.shape)



97286 [0 1] [48643 48643]
(97286, 72)


In [26]:
## Separating Predicted vs Target Values 
predictions = pipeline.predict(x_test)
target = y_test

pipeline.score(x_test, y_test)


0.9999383261723167

In [27]:
# Confusion Matrix from your predicted values 
from sklearn.metrics import *

# Passing Target and Prediction Values to Confusion_Matrix
CM = confusion_matrix(target, predictions)
print ("\n\nConfusion matrix:\n", CM)

tn, fp, fn, tp = CM.ravel()
print ("\nTP, TN, FP, FN:", tp, ",", tn, ",", fp, ",", fn)


P = precision_score(target, predictions)
print ("\nPrecision Score:", np.round(P, 2))
print (' ')
R = recall_score(target, predictions)
print ("\nRecall Score:", np.round(R, 2))
print (' ')
F1 = f1_score(target, predictions)
print ("\nF1 Score:", np.round(F1, 2))
print (' ')
a = accuracy_score(target, predictions)
print ("\nAccuracy Score:", np.round(a, 2))
print (' ')





Confusion matrix:
 [[24387     3]
 [    0 24253]]

TP, TN, FP, FN: 24253 , 24387 , 3 , 0

Precision Score: 1.0
 

Recall Score: 1.0
 

F1 Score: 1.0
 

Accuracy Score: 1.0
 


#### Summary for Part 2
The confusion matrix metrics after resampling data shows improved results. Precision score is at now at 100%. The Recall score has increased to 100%. Both scores shows a large increase. Previously the accuracy score could not be used since it was not as reliable with class imbalanced - however now we see that there were 24390 negative Class values and 24387 positive Class values which is much more closer to each other than when there was an imbalance. 

## Conclusion:

The purpose of this assignment was to build a machine learning algorithm that can accurately identify "bad" and "good" or normal connections on a network. One of the challenging aspects of this dataset was the class imbalance, regarding how many available values were "good" connections vs how many available values were "bad". There was a disproportionate amount of "good" values in the data, which makes the model very good at only detecting "good" connections. One way to combat this issue is by resampling the data so that there are even amounts of "good" and "bad" connections - and the model can become more accurate detecting both classes. 